In [ ]:
import ee
ee.Initialize()
import pandas as pd

In [ ]:
tile_file = "gee_tile_select.csv"
data = pd.read_csv(tile_file)
minx = data['left']
maxy = data['top']
maxx = data['right']
miny = data['bottom']
tile_id =data["id"]
size_data = len(minx)

In [ ]:
def start_process(i):
    #define start and end date, extent
    s_date = '2015-01-01'
    e_date = '2016-01-01'
    area = ee.Geometry.Rectangle([minx[i], maxy[i], maxx[i], miny[i]])
    tile_name = 'plantation_' + str(tile_id[i])
    
    #entropy
    #Define a neighborhood with a kernel.
    entropy_args = {'radius': 4}
    square = ee.Kernel.square(**entropy_args)
    
    #define image objects
    gain = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('gain')
    lossyear = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('lossyear')

    
    ##Sentinel band processing
    ##Filter to get images with VV and VH dual polarization.
    ##Filter to get images collected in interferometric wide swath mode.
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.date(s_date, e_date))
    vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW'))
    vhAscending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    vhDescending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    sentinel1_VH = ee.ImageCollection(vhAscending.select('VH').merge(vhDescending.select('VH'))).mean()
    sentinel1_VV = ee.ImageCollection(vhAscending.select('VV').merge(vhDescending.select('VV'))).mean()
    sentinel1_VH_entropy = sentinel1_VH.int().entropy(square)
    
    zones = gain.eq(1).And((lossyear.lt(12)).Or(lossyear.gt(16)))
    zones = zones.updateMask(zones)

    #reduce raster to polygons
    reduce_combine_args = {
      'reducer2': ee.Reducer.stdDev(),
      'sharedInputs': True
    }
    reducers = ee.Reducer.mean().combine(**reduce_combine_args);
    
    reduce_args = {
      'geometry': area,
      'crs': gain.projection(),
      'scale': 100,
      'geometryType': 'polygon',
      'eightConnected': True,
      'labelProperty': 'regrowth',
      'reducer': reducers,
      'bestEffort': True,
      'maxPixels': 1e18
    }

    #sentinel1 bands
    vectors = zones.addBands(sentinel1_VH).addBands(sentinel1_VV).addBands(sentinel1_VH_entropy).reduceToVectors(**reduce_args)
   
    featureCollection = ee.FeatureCollection([vectors]).flatten()
    
    down_args = {
    'collection': featureCollection,
    'folder': 'plantation_sentinel1_2015_v3',
    'description': tile_name,
    'fileFormat': 'KML'
    }
    task = ee.batch.Export.table.toDrive(**down_args)
    task.start()

In [1]:
#loop through tiles
for i in range(1,size_data):
    start_process(i)